### Render a Texture

In this example we will create a checkboard texture and display it on a plane on the gound

In [1]:
import ipywebgl
import numpy as np

In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.execute_commands(execute_once=True)

In [3]:
program = w.create_program_ext(
"""#version 300 es

//the ViewBlock that is automatically filled by ipywebgl
layout(std140) uniform ViewBlock
{
    mat4 u_cameraMatrix;          //the camera matrix in world space
    mat4 u_viewMatrix;            //the inverse of the camera matrix
    mat4 u_projectionMatrix;      //the projection matrix
    mat4 u_viewProjectionMatrix;  //the projection * view matrix
};

in vec2 in_vert;
in vec2 in_texcoord;
out vec2 v_texcoord;
void main() {
   gl_Position = u_viewProjectionMatrix * vec4(in_vert.x, 0.0, in_vert.y, 1.0);
   v_texcoord = in_texcoord;
}
"""
,
"""#version 300 es
precision highp float;
uniform sampler2D u_texture;
in vec2 v_texcoord;
out vec4 f_color;
void main() {
    f_color = texture(u_texture, v_texcoord);
}
""")


In [7]:
vbo = w.create_buffer_ext(
    #x z u v
    src_data=np.array([
        -100, -100, 0, 10,
        100, -100, 10, 10,
        100, 100, 10, 0,
        -100, 100, 0, 0,
    ], dtype=np.float32)
)

vao = w.create_vertex_array_ext(
    program,
    [
        (vbo, '2f32 2f32', 'in_vert', 'in_texcoord'),
    ]
)

### Create a 2 by 2 texture

simple rbga 2 by 2 texture

In [5]:
tex = w.create_texture()
w.active_texture(0)
w.bind_texture('TEXTURE_2D', tex)
w.tex_image_2d('TEXTURE_2D', 0, 'RGBA', 2, 2, 0, 'RGBA', 'UNSIGNED_BYTE',
              np.array([120,120,120,255, 180,180,180,255, 180,180,180,255, 120,120,120,255], dtype=np.uint8))
w.execute_commands(execute_once=True)

Bind the texture with a repeat pattern, because our UV's are from 0 to 10 and not 0 to 1

In [8]:
w.clear()
w.use_program(program)
w.active_texture(0)
w.bind_texture('TEXTURE_2D', tex)
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MAG_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_MIN_FILTER', 'NEAREST')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_S', 'REPEAT')
w.tex_parameter('TEXTURE_2D', 'TEXTURE_WRAP_T', 'REPEAT')
w.bind_vertex_array(vao)
w.draw_arrays('TRIANGLE_FAN', 0, 4)
w.execute_commands()
w

GLViewer(camera_pitch=-26.2, camera_pos=[17.809647922495124, 27.95317454386815, 132.78695031297934], camera_ya…